**This notebook contains a script that creates student submission folders for SETA**

**Add Local Library to Path**

In [66]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [67]:
import pandas as pd
import re
from local_library import import_worksheet
from local_library import export_worksheet

**Import Data**

In [68]:
#import cohort data
cohort_df = import_worksheet("Cohort 2020 Data", "Cohort 2020 Data")

**Create Local Library**

In [69]:
def list_files(folder_id):
    #list all files in a google folder
    #params:
    #       folder_id: the google id of the parent folder
    #return:
    #       file_list: a list of files in the parent folder
    
    file_list = drive.ListFile(
        {
            'q': "'{}' in parents and trashed=false".format(folder_id),
            'corpora': "teamDrive",
            'teamDriveId': "0AL2boqVPcgjKUk9PVA",
            'includeTeamDriveItems': "true",
            'supportsTeamDrives': "true"
        }
    ).GetList()
    return file_list


def create_folder(parent_folder_id, folder_name, email=''):
    #create folder if it does not exist
        #params:
    #       parent_folder_id: the google id of the parent folder
    #       folder_name: the name of the folder created
    #return:
    #       folder_id: the google id if the created folder

    file_list = list_files(parent_folder_id)
    folder_id = ''
    for folder in file_list:
        if folder['title'] == folder_name:
            return folder['id']
    if not folder_id:
        folder = drive.CreateFile({'parents': [{'id': parent_folder_id}], 'title': folder_name,
                                   'mimeType': 'application/vnd.google-apps.folder'})
        folder.Upload(param={'supportsTeamDrives': True})
        
        if email:
            share_folder(folder, email)

        return folder['id']
    return False


def share_folder(folder, email):
    new_permission = {
        'type': 'user',
        'value': email,
        'role': 'writer'
    }
    permission = folder.auth.service.permissions().insert(
        fileId=folder['id'], body=new_permission, supportsTeamDrives=True).execute(http=folder.http)
        

**Update Columns Names**

In [70]:

column_names = {"Firstname":"learner_name", "Surname": "learner_surname",
                "Mobile Number":"learner_phone", "Student Email":"learner_email",
                "Middle Name/s":"middle_name"
               }

cohort_df = cohort_df.rename(columns=column_names)

**Create Fullname**

In [71]:
cohort_df['middle_name'] = cohort_df['middle_name'].fillna("")
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['learner_surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()


**Subset Useful Columns and Rows**

In [72]:
column_names = ['Username', 'learner_email', 'learner_fullname']

cohort_df = cohort_df[cohort_df['Active / Not active/Early absorption'] == 'Active']
cohort_df = cohort_df[column_names]

In [73]:
cohort_df['folder_id'] = ''
len(cohort_df)

257

**Create Folders**

In [85]:
for index, row in cohort_df.iterrows():
    parent_id = create_folder("1-2tkFz7qrd2MD-vvudm0AlyK46YFt5so",row['learner_fullname'], row['learner_email'])
    create_folder(parent_id, "video presentations")
    create_folder(parent_id, "video demonstrations")
    create_folder(parent_id, "technical reports")
    cohort_df.loc[cohort_df['learner_fullname'] == row['learner_fullname'], 'folder_id'] = parent_id


In [86]:
export_worksheet("POE Compilation","Evidence Submission", cohort_df)

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)